In [24]:
import logging
from typing import List

import pandas as pd
from langchain.chat_models import AzureChatOpenAI
from langchain.chat_models.base import BaseChatModel
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.evaluation import EmbeddingDistance
from langchain.evaluation import load_evaluator, EvaluatorType
from langchain.evaluation.schema import StringEvaluator
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [3]:
loader = TextLoader("../state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [4]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [5]:
hf_embeddings_vector_db = FAISS.from_documents(docs, hf_embeddings)
hf_embeddings_vector_db.save_local("faiss_index")

In [6]:
question = "What did the president say about economics?"
docs = hf_embeddings_vector_db.similarity_search(question)
docs[0].page_content

'We’re going after the criminals who stole billions in relief money meant for small businesses and millions of Americans.  \n\nAnd tonight, I’m announcing that the Justice Department will name a chief prosecutor for pandemic fraud. \n\nBy the end of this year, the deficit will be down to less than half what it was before I took office.  \n\nThe only president ever to cut the deficit by more than one trillion dollars in a single year. \n\nLowering your costs also means demanding more competition. \n\nI’m a capitalist, but capitalism without competition isn’t capitalism. \n\nIt’s exploitation—and it drives up prices. \n\nWhen corporations don’t have to compete, their profits go up, your prices go up, and small businesses and family farmers and ranchers go under. \n\nWe see it happening with ocean carriers moving goods in and out of America. \n\nDuring the pandemic, these foreign-owned companies raised prices by as much as 1,000% and made record profits.'

In [7]:
gpt35_azure_llm = AzureChatOpenAI(
    temperature=0,
    openai_api_key="02e3dbabaf334ccb959cbeadbd3f99c3",
    openai_api_base="https://llm-x-gpt.openai.azure.com/",
    deployment_name='LLM-X-GPT35-TURBO',
    openai_api_version="2023-03-15-preview"
)

In [8]:
gpt4_azure_llm = AzureChatOpenAI(
    temperature=0,
    openai_api_key="a8d69f68a36b40789df2cc3fdbaacda9",
    openai_api_base="https://llmx-gpt-canada-east.openai.azure.com/",
    deployment_name='LLM-X-GPT-4',
    openai_api_version="2023-03-15-preview"
)

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings

openai_azure_embeddings = OpenAIEmbeddings(
    openai_api_key="02e3dbabaf334ccb959cbeadbd3f99c3",
    openai_api_base="https://llm-x-gpt.openai.azure.com/",
    deployment_name='LLM-X-Embedding'
)

/home/watson/.virtualenvs/QnAEvaluation/lib/python3.9/site-packages/langchain/embeddings/openai.py:214: UserWarning: WARNING! deployment_name is not default parameter.
                    deployment_name was transferred to model_kwargs.
                    Please confirm that deployment_name is what you intended.
  warnings.warn(


In [10]:
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=hf_embeddings_vector_db.as_retriever(),
    llm=gpt35_azure_llm
)

unique_docs = retriever_from_llm.get_relevant_documents(query=question)

INFO:langchain.retrievers.multi_query:Generated queries: ["1. Can you provide any information on the president's statements regarding the field of economics?", "2. I'm interested in knowing the president's views and comments on the subject of economics. Could you share any relevant information?", '3. Could you please share any insights or remarks made by the president in relation to economics?']


In [11]:
from langchain.chains import RetrievalQA

qa_rag_chain = RetrievalQA.from_chain_type(
    llm=gpt35_azure_llm,
    retriever=hf_embeddings_vector_db.as_retriever()
)

question = "What did the president say about Zelensky?"
qa_rag_chain({"query": question})

{'query': 'What did the president say about Zelensky?',
 'result': "The president mentioned President Zelenskyy of Ukraine in the context of the Ukrainian people's resistance against Russia's invasion. He praised their fearlessness, courage, and determination, which he said inspires the world. However, he did not provide specific details about Zelenskyy's actions or policies."}

In [12]:
from langchain.chains import RetrievalQA

qa_rag_chain = RetrievalQA.from_chain_type(
    llm=gpt35_azure_llm,
    retriever=hf_embeddings_vector_db.as_retriever()
)

question = "What did the president say about Zelensky?"
qa_rag_chain({"query": question})

{'query': 'What did the president say about Zelensky?',
 'result': 'The president mentioned President Zelenskyy of Ukraine and praised the fearlessness, courage, and determination of the Ukrainian people.'}

In [13]:
from langchain.chains import QAGenerationChain
from langchain import PromptTemplate

templ = """You are a smart assistant designed to help high school teachers come up with reading comprehension questions.
Given a piece of text, you must come up with a {k} different question and answer pairs that can be used to test a student's reading comprehension abilities.
When coming up with this question/answer pair, each pair must be respond in the following format:

{{
    "question": "$YOUR_QUESTION_HERE",
    "answer": "$THE_ANSWER_HERE"
}}

So in your final answer you should response with a list of {k} pairs in this format:

```
[{{
    "question": "$YOUR_QUESTION_HERE",
    "answer": "$THE_ANSWER_HERE"
}},
 {{
    "question": "$YOUR_QUESTION_HERE",
    "answer": "$THE_ANSWER_HERE"
}},
 {{
    "question": "$YOUR_QUESTION_HERE",
    "answer": "$THE_ANSWER_HERE"
    }}
]
```

Please come up with a list of {k} question/answer pairs, in the specified list of JSONS format, for the following text:
----------------
{text}
"""

multi_qa_prompt_gpt35 = PromptTemplate.from_template(template=templ, partial_variables={"k": 5})
qa_generation_chain_gpt35 = QAGenerationChain.from_llm(llm=gpt35_azure_llm, prompt=multi_qa_prompt_gpt35)

In [29]:
from langchain.chains import QAGenerationChain
from langchain import PromptTemplate

multi_qa_prompt_template = """You are a smart assistant designed to help high school teachers come up with reading comprehension questions.
Given a piece of text, you must come up with a {k} different question and answer pairs that can be used to test a student's reading comprehension abilities.
When coming up with this question/answer pair, each pair must be respond in the following format:

{{
    "question": "$YOUR_QUESTION_HERE",
    "answer": "$THE_ANSWER_HERE"
}}
Please come up with a list of {k} question/answer pairs, in the specified list of dict, for the following text:
----------------
{text}
"""

In [58]:
def create_qna_GT_df(docs: List[Document], num_of_qna_for_doc: int) -> pd.DataFrame:
    multi_qa_prompt = PromptTemplate.from_template(template=multi_qa_prompt_template,
                                                   partial_variables={"k": num_of_qna_for_doc})
    qa_generation_chain = QAGenerationChain.from_llm(llm=gpt4_azure_llm, prompt=multi_qa_prompt)

    qna_GT = []
    for doc in docs:
        doc_qna = qa_generation_chain.run(doc.page_content)[0]
        qna_GT += doc_qna

    qna_GT_df = pd.DataFrame(qna_GT)
    return qna_GT_df

In [35]:
def get_qna_with_chain_answers_df(qa_rag_chain: RetrievalQA, qna_GT_df: pd.DataFrame) -> pd.DataFrame:
    qna_with_chain_answers = qna_GT_df.copy()
    qna_with_chain_answers["chain_answer"] = qna_with_chain_answers.apply(
        lambda qna: qa_rag_chain({"query": qna["question"]})["result"], axis=1)

    return qna_with_chain_answers

In [36]:
def get_evaluator_score(evaluator: StringEvaluator, qna_with_chain_answer: pd.Series) -> float:
    grade = evaluator.evaluate_strings(
        prediction=qna_with_chain_answer["chain_answer"],
        reference=qna_with_chain_answer["answer"],
        input=qna_with_chain_answer["question"])

    return grade["score"]

In [37]:
def get_grades_for_chain_qna(qna_with_chain_answers_df: pd.DataFrame) -> pd.DataFrame:
    grades_for_chain_qna = qna_with_chain_answers_df.copy()

    labeled_criteria_evaluator = load_evaluator(evaluator=EvaluatorType.LABELED_CRITERIA,
                                                criteria="correctness",
                                                llm=gpt4_azure_llm)

    embedding_distance_evaluator = load_evaluator(evaluator=EvaluatorType.EMBEDDING_DISTANCE,
                                                  distance_metric=EmbeddingDistance.COSINE,
                                                  embeddings=hf_embeddings,
                                                  llm=gpt4_azure_llm)

    qa_llm__jugde_evaluator = load_evaluator(evaluator=EvaluatorType.QA,
                                             llm=gpt4_azure_llm)

    grades_for_chain_qna["labeled_criteria_grades"] = qna_with_chain_answers_df.apply(
        lambda qna_with_chain_answer: get_evaluator_score(
            evaluator=labeled_criteria_evaluator,
            qna_with_chain_answer=qna_with_chain_answer), axis=1)

    grades_for_chain_qna["embedding_distance_grades"] = qna_with_chain_answers_df.apply(
        lambda qna_with_chain_answer: get_evaluator_score(
            evaluator=embedding_distance_evaluator,
            qna_with_chain_answer=qna_with_chain_answer), axis=1)

    grades_for_chain_qna["qa_llm_jugde_grades"] = qna_with_chain_answers_df.apply(
        lambda qna_with_chain_answer: get_evaluator_score(
            evaluator=qa_llm__jugde_evaluator,
            qna_with_chain_answer=qna_with_chain_answer), axis=1)

    return grades_for_chain_qna

In [40]:
def run_evaluation(qa_rag_chain: RetrievalQA, qna_GT_df: pd.DataFrame) -> pd.DataFrame:
    qna_with_chain_answers_df = get_qna_with_chain_answers_df(qa_rag_chain, qna_GT_df)
    grades_for_chain_qna = get_grades_for_chain_qna(qna_with_chain_answers_df)
    return grades_for_chain_qna

In [60]:
qna_GT_df = create_qna_GT_df(docs=docs, num_of_qna_for_doc=5)
run_evaluation(qa_rag_chain=qa_rag_chain, qna_GT_df=qna_GT_df)

/home/watson/.virtualenvs/QnAEvaluation/lib/python3.9/site-packages/langchain/evaluation/schema.py:104: UserWarning: Ignoring input in EmbeddingDistanceEvalChain, as it is not expected.
  warn(self._skip_input_warning)


,question,answer,chain_answer,labeled_criteria_grades,embedding_distance_grades,qa_llm_jugde_grades
0,What action is being taken against the crimina...,The Justice Department will name a chief prose...,The Justice Department is naming a chief prose...,1,8.552997e-02,1
1,What will be the state of the deficit by the e...,"By the end of this year, the deficit will be d...",I don't have that information.,0,9.877543e-01,0
2,What is the speaker's view on capitalism witho...,The speaker believes that capitalism without c...,The speaker's view is that capitalism without ...,1,1.377589e-01,1
3,What happens when corporations don't have to c...,"When corporations don’t have to compete, their...","When corporations don't have to compete, their...",1,2.166901e-01,1
4,What did foreign-owned companies do during the...,"During the pandemic, these foreign-owned compa...","During the pandemic, foreign-owned companies r...",1,9.677858e-03,1
5,What is the author's plan to fight inflation?,The author's plan to fight inflation includes ...,The author's plan to fight inflation includes ...,0,6.139819e-02,1
6,What does the author suggest to do instead of ...,The author suggests making goods in America in...,The author suggests that instead of relying on...,1,3.048098e-01,1
7,What do economists call the author's plan?,Economists call the author's plan 'increasing ...,"Economists call the author's plan ""increasing ...",1,1.081692e-02,1
8,What is the first step of the author's plan?,The first step of the author's plan is to cut ...,The first step of the author's plan is to cut ...,1,1.110223e-16,1
9,Who does the author mention meeting in Virginia?,The author mentions meeting a 13-year-old boy ...,The author does not mention meeting anyone in ...,0,5.695484e-01,0
